# Word Embeddings with WMD

In [ ]:
###command line setup
#
#source venv/bin/activate
#pip install ipykernel
#python -m ipykernel install --user --name=gintastic
#pip install -r requirements.txt

#pip freeze > requirements.txt

In [ ]:
##TODO 
## Instead of averaging the work embeddigns to get the docs
##DO WBD on each word embedding. 
#   
##""""""Another way to do this unsupervised would be to check out word mover's distance for word embeddings. 
##We haven't implemented this in Flair but there are libraries out there that do this for you. 
##I could imagine that this approach works better than simple averaging of word vectors.""""""
#
#https://markroxor.github.io/gensim/static/notebooks/WMD_tutorial.html
#https://towardsdatascience.com/word-movers-distance-for-text-similarity-7492aeca71b0
#https://medium.com/@stephenhky/word-movers-distance-as-a-linear-programming-problem-6b0c2658592e
#https://towardsdatascience.com/word-distance-between-word-embeddings-cc3e9cf1d632
#https://vene.ro/blog/word-movers-distance-in-python.html
#https://github.com/src-d/wmd-relax

In [28]:
import pandas as pd
import numpy as np
import pickle
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
import torch

In [29]:
file_name = 'gin_reviews.csv'
df = pd.read_csv(file_name, index_col=0)
print(df.shape)
df.head(3)

(112, 2)


,description drizly,description caskers
gin,,
tommyrotter american gin,Pot-distilled in small batches to articulate a...,This impassioned take on an American gin is po...
beefeater london dry gin,"When only a tried-and-true dry gin will do, tu...","Since 1863, Beefeater Gin has remained one of ..."
rabbit hole london dry gin,NaN,This incredible gin has an exclusive distincti...


In [30]:
df.tail(3)

,description drizly,description caskers
gin,,
deviation citrus rose gin,"As with our American single malt whiskey, Citr...",NaN
deviation mountain herb gin,Mountain Herb Gin is Crafted from the same fou...,NaN
deviation spice trade gin,"Open fermentation in custom wooden vats, follo...",NaN


In [31]:
from flair.embeddings import WordEmbeddings, CharacterEmbeddings

# init standard GloVe embedding
glove_embedding = WordEmbeddings('glove')

# init Flair forward and backwards embeddings
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

2020-02-23 17:12:38,608 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim.vectors.npy not found in cache, downloading to /tmp/tmp4rm8dtzc


100%|██████████| 160000128/160000128 [00:11<00:00, 14376806.62B/s]

2020-02-23 17:12:50,226 copying /tmp/tmp4rm8dtzc to cache at /home/ec2-user/.flair/embeddings/glove.gensim.vectors.npy
2020-02-23 17:12:50,384 removing temp file /tmp/tmp4rm8dtzc


2020-02-23 17:12:50,876 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/glove.gensim not found in cache, downloading to /tmp/tmpz53j8uiq


100%|██████████| 21494764/21494764 [00:01<00:00, 11499290.33B/s]

...

100%|██████████| 73034624/73034624 [00:05<00:00, 13642980.50B/s]

2020-02-23 17:13:00,974 copying /tmp/tmpp1ofeslu to cache at /home/ec2-user/.flair/embeddings/news-forward-0.4.1.pt
2020-02-23 17:13:01,048 removing temp file /tmp/tmpp1ofeslu


2020-02-23 17:13:21,059 https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings-v0.4.1/big-news-backward--h2048-l1-d0.05-lr30-0.25-20/news-backward-0.4.1.pt not found in cache, downloading to /tmp/tmp9abud6vu


100%|██████████| 73034575/73034575 [00:05<00:00, 12665741.24B/s]

2020-02-23 17:13:27,341 copying /tmp/tmp9abud6vu to cache at /home/ec2-user/.flair/embeddings/news-backward-0.4.1.pt
2020-02-23 17:13:27,415 removing temp file /tmp/tmp9abud6vu


In [32]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

# create a StackedEmbedding object that combines glove and forward/backward flair embeddings
stacked_embeddings = StackedEmbeddings([
                                        glove_embedding,
                                        flair_embedding_forward,
                                        flair_embedding_backward,
                                       ])

In [33]:
sentence = Sentence('The grass is green.')

# just embed a sentence using the StackedEmbedding as you would with any single embedding.
stacked_embeddings.embed(sentence)

# now check out the embedded tokens.
for word in sentence:
    print(word)
    print(word.embedding)

Token: 1 The
tensor([-0.0382, -0.2449,  0.7281,  ..., -0.0056, -0.0066,  0.0187],
       device='cuda:0')
Token: 2 grass
tensor([-0.8135,  0.9404, -0.2405,  ...,  0.0363, -0.0245, -0.0144],
       device='cuda:0')
Token: 3 is
tensor([-5.4264e-01,  4.1476e-01,  1.0322e+00,  ...,  1.4566e-04,
        -9.3342e-03, -4.7046e-03], device='cuda:0')
Token: 4 green.
tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  4.4976e-05,
        -1.0650e-01,  1.5009e-02], device='cuda:0')


In [ ]:
#TODO. implement WBD DIY
following this
https://vene.ro/blog/word-movers-distance-in-python.html

In [34]:
#first word
word1 = sentence[0] 
v_1 = word1.embedding.numpy()

#second word
word2 = sentence[1] 
v_2 = word2.embedding.numpy()

TypeError: can't convert CUDA tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
#TODO implement DIU WMD on the sentence class using matric calcs with pytorch tensors. 

## Document Embeddings with cosine similarity

In [35]:
import pandas as pd
import numpy as np
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, Sentence
import torch

In [36]:
# initialize the word embeddings
#glove_embedding = WordEmbeddings('glove')
flair_embedding_forward = FlairEmbeddings('news-forward')
flair_embedding_backward = FlairEmbeddings('news-backward')

# initialize the document embeddings, mode = mean
make_document_embeddings = DocumentPoolEmbeddings([flair_embedding_backward,
                                                   flair_embedding_forward])

In [37]:
def get_glove_document_embeddings(document):
    #create an example sentence
    doc = Sentence(document)

    # embed the sentence with our document embedding
    make_document_embeddings.embed(doc)

    # now check out the embedded sentence.
    return doc.get_embedding()

In [38]:
query = "I want a whiskey that tastes like pencil shavings and pepper. Add seem caramel sauce on top"
query = "I am looking for a blended scottch with a rich oaky flavor. I like spicy and fruit and raisins."
print(query+"\n")
query_embedding = get_glove_document_embeddings(query)

doc1 = df.loc[0].description
print(doc1+"\n")
doc1_embedding = get_glove_document_embeddings(doc1)

doc2 = df.loc[3].description
print(doc2)
doc2_embedding = get_glove_document_embeddings(doc2)

I am looking for a blended scottch with a rich oaky flavor. I like spicy and fruit and raisins.



TypeError: cannot do label indexing on <class 'pandas.core.indexes.base.Index'> with these indexers [0] of <class 'int'>

In [39]:
def get_similarity(query, doc):
    cosine_sim = torch.nn.CosineSimilarity(dim=0)(query, doc)
    return cosine_sim

In [40]:
print(get_similarity(query_embedding, doc1_embedding))
print(get_similarity(query_embedding, doc2_embedding))
print(get_similarity(doc1_embedding, doc2_embedding))

NameError: name 'doc1_embedding' is not defined

In [41]:
#selecting the right similarity metric
doc1 = "Obama speaks to the media in Illinois"
print(doc1+"\n")
doc1_embedding = get_glove_document_embeddings(doc1)

doc2 = "The President greets the press in Chicago"
print(doc2+"\n")
doc2_embedding = get_glove_document_embeddings(doc2)

print(get_similarity(doc1_embedding, doc2_embedding))
print(get_similarity(doc1_embedding, doc2_embedding))

Obama speaks to the media in Illinois

The President greets the press in Chicago

tensor(0.7136, device='cuda:0', grad_fn=<DivBackward0>)
tensor(0.7136, device='cuda:0', grad_fn=<DivBackward0>)
